# 🧬 Full Variant Calling Pipeline (FASTQ → VCF)

This section walks you through the entire process of generating a VCF file from raw sequencing reads (FASTQ format).

## 📁 Step 2: Prepare Input Files
Make sure you have:
- `reads_1.fastq` and `reads_2.fastq` (paired-end FASTQ)
- `reference.fasta` (reference genome)


## 🧪 Step 3: Quality Control 


In [1]:
!fastqc -f fastq STP_workshop/SRR3184278_1.fastq STP_workshop/SRR3184278_2.fastq

null
null
Started analysis of SRR3184278_1.fastq
Approx 5% complete for SRR3184278_1.fastq
Approx 10% complete for SRR3184278_1.fastq
Approx 15% complete for SRR3184278_1.fastq
Approx 20% complete for SRR3184278_1.fastq
Approx 25% complete for SRR3184278_1.fastq
Approx 30% complete for SRR3184278_1.fastq
Approx 35% complete for SRR3184278_1.fastq
Approx 40% complete for SRR3184278_1.fastq
Approx 45% complete for SRR3184278_1.fastq
Approx 50% complete for SRR3184278_1.fastq
Approx 55% complete for SRR3184278_1.fastq
Approx 60% complete for SRR3184278_1.fastq
Approx 65% complete for SRR3184278_1.fastq
Approx 70% complete for SRR3184278_1.fastq
Approx 75% complete for SRR3184278_1.fastq
Approx 80% complete for SRR3184278_1.fastq
Approx 85% complete for SRR3184278_1.fastq
Approx 90% complete for SRR3184278_1.fastq
Approx 95% complete for SRR3184278_1.fastq
Analysis complete for SRR3184278_1.fastq
Started analysis of SRR3184278_2.fastq
Approx 5% complete for SRR3184278_2.fastq
Approx 10% co

## 🧬 Step 4: Index the Reference Genome

In [2]:
!bwa index STP_workshop/salmonella_ref.fasta

[bwa_index] Pack FASTA... 0.05 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.73 seconds elapse.
[bwa_index] Update BWT... 0.02 sec
[bwa_index] Pack forward-only FASTA... 0.02 sec
[bwa_index] Construct SA from BWT and Occ... 0.24 sec
[main] Version: 0.7.19-r1273
[main] CMD: bwa index STP_workshop/salmonella_ref.fasta
[main] Real time: 1.125 sec; CPU: 1.082 sec


## 🔗 Step 5: Align Reads to Reference

In [3]:
!bwa mem STP_workshop/salmonella_ref.fasta STP_workshop/SRR3184278_1.fastq STP_workshop/SRR3184278_2.fastq > STP_workshop/aln.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 74152 sequences (10000267 bp)...
[M::process] read 74058 sequences (10000037 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (21, 33687, 5, 10)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (148, 237, 546)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 1342)
[M::mem_pestat] mean and std.dev: (298.00, 181.20)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1740)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (136, 249, 417)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 979)
[M::mem_pestat] mean and std.dev: (295.99, 199.87)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1260)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] analyzing insert size dist

## 📚 Step 6: Sort and Index BAM File

In [4]:
!samtools view -Sb STP_workshop/aln.sam > STP_workshop/aln.bam
!samtools sort STP_workshop/aln.bam -o STP_workshop/aln.sorted.bam
!samtools index STP_workshop/aln.sorted.bam

## 🧬 Step 7: Variant Calling with GATK

In [ ]:
!bcftools mpileup -Ou -f STP_workshop/salmonella_ref.fasta STP_workshop/aln.sorted.bam 
!bcftools call -mv -Oz -o STP_workshop/variants.vcf.gz
!bcftools index STP_workshop/variants.vcf.gz
!bcftools view STP_workshop/variants.vcf.gz | less -S

[E::fai_load3_core] Failed to open FASTA file STP_workshop/salmonella_ref.fasta

About:   SNP/indel variant calling from VCF/BCF. To be used in conjunction with bcftools mpileup.
         This command replaces the former "bcftools view" caller. Some of the original
         functionality has been temporarily lost in the process of transition to htslib,
         but will be added back on popular demand. The original calling model can be
         invoked with the -c option.
Usage:   bcftools call [options] <in.vcf.gz>

File format options:
       --no-version                Do not append version and command line to the header
   -o, --output FILE               Write output to a file [standard output]
   -O, --output-type b|u|z|v       Output type: 'b' compressed BCF; 'u' uncompressed BCF; 'z' compressed VCF; 'v' uncompressed VCF [v]
   -O, --output-type u|b|v|z[0-9]  u/b: un/compressed BCF, v/z: un/compressed VCF, 0-9: compression level [v]
       --ploidy ASSEMBLY[?]        Predefined p

## 🧪 Step 8: Output VCF File
You should now have a file called `variants.vcf.gz` containing SNPs and Indels.
You can open it with any text viewer or parse it using Python or VEP.

## 🧪 Step 9: Variant annotation

In [ ]:
!snpeff download Salmonella_enterica
!snpeff Salmonella_enterica STP_workshop/variants.vcf.gz > STP_workshop/annotated.vcf

## ✅ Summary

- Identified genomic variants using bcftool
- Annotated variants using snpEFF, ANNOVAR or Ensemble VEP